In [38]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [39]:
data=pd.read_csv('Ginzberg.csv', usecols=['simplicity','fatalism','depression'])
data.head()

,simplicity,fatalism,depression
0,0.92983,0.35589,0.59870
1,0.91097,1.18439,0.72787
2,0.53366,-0.05837,0.53411
3,0.74118,0.35589,0.56641
4,0.53366,0.77014,0.50182


In [40]:
data.shape

(82, 3)

In [41]:
train, test= train_test_split(data, test_size=0.20)

In [42]:
train.shape, test.shape

((65, 3), (17, 3))

In [43]:
x_train=train[['simplicity','fatalism']]
x_train.shape

(65, 2)

In [44]:
y_train=train[['depression']]

In [45]:
x_test=test[['simplicity','fatalism']]

In [46]:
y_test=test[['depression']]

In [47]:
seed=7
np.random.seed(seed)

In [48]:
regr=linear_model.LinearRegression()


In [49]:
model=regr.fit(x_train, y_train)

In [50]:
regr.intercept_, regr.coef_

(array([ 0.20999386]), array([[ 0.34578906,  0.44951552]]))

In [51]:
trainpred=model.predict(x_test)

In [52]:
r_linear=r2_score(y_test, trainpred)

In [53]:
print(r_linear)

0.504295474662


**NEURAL NETWORK**

In [54]:
import numpy as np
import pandas as pd
from keras.layers import Dense
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [55]:
dataset=data.values

In [56]:
dataset.shape

(82, 3)

In [57]:
x=dataset[:,0:2]
y=dataset[:,2]

In [58]:
x.shape, y.shape

((82, 2), (82,))

In [59]:
seed=7
np.random.seed(seed)

In [60]:
def basic_model():
    model=Sequential()
    model.add(Dense(2, input_dim=2, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [61]:
#standardise the input data
estimators=[]
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=basic_model, nb_epoch=50, batch_size=5, verbose=0)))
pipeline=Pipeline(estimators)
#evaluate the model
kfold=KFold(n_splits=10, shuffle=True, random_state=seed)
results=cross_val_score(pipeline, x, y, cv=kfold)
print('r square statistic', results.mean(), results.std())

C:\Users\saiva\Miniconda3\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, activation="relu", input_dim=2, kernel_initializer="normal")`
  app.launch_new_instance()
C:\Users\saiva\Miniconda3\lib\site-packages\ipykernel\__main__.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`
C:\Users\saiva\Miniconda3\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, activation="relu", input_dim=2, kernel_initializer="normal")`
  app.launch_new_instance()
C:\Users\saiva\Miniconda3\lib\site-packages\ipykernel\__main__.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`
C:\Users\saiva\Miniconda3\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, activation="relu", input_dim=2, kernel_initializer="normal")`
  app.launch_

r square statistic 0.910646432079 0.24171661744


**NEURAL NETWORK WITH TENSORFLOW**

In [27]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [28]:
COLUMNS=['simplicity', 'fatalism', 'depression']
FEATURES=['simplicity','fatalism']
LABELS=['depression']

In [29]:
training_set=pd.read_csv('Ginzberg.csv', usecols=['simplicity','fatalism','depression'])

In [30]:
feature_cols=[tf.feature_column.numeric_column(k) for k in FEATURES]

In [31]:
regressor=tf.estimator.DNNRegressor(feature_columns=feature_cols, hidden_units=[4,4])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\saiva\\AppData\\Local\\Temp\\tmpb163ak8u', '_log_step_count_steps': 100, '_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_save_checkpoints_steps': None}


In [32]:
def get_input_fn(dataset, num_epochs=None, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(x=pd.DataFrame({k:dataset[k].values for k in FEATURES}), 
                                               y=pd.Series(dataset[LABELS].values),
                                              num_epochs=num_epochs, shuffle=shuffle)

In [33]:
regressor.train(input_fn=get_input_fn(training_set), steps=5000)

Exception: Data must be 1-dimensional